In [1]:
# Necessary Imports
import os
import torch
import pandas as pd
from PIL import Image
from torchvision.models import mobilenet_v2
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

# Step 1: Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 2: Define the Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [(os.path.join(root_dir, f), f) for f in os.listdir(root_dir)]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, image_id = self.image_files[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, image_id
        except (OSError, IOError) as e:
            print(f"Warning: Skipping corrupted image {img_path}. Error: {e}")
            return self.__getitem__((idx + 1) % len(self))  # Return the next valid image

# Step 3: Define Transforms
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step 4: Load the Model
state_dict_path = '/kaggle/input/mobilenet_v2/other/default/1/mobilenetv2_epoch_5.pth'  # Update with your model path

# Load the state dictionary
state_dict = torch.load(state_dict_path, map_location=device)

# Remove 'module.' prefix from state_dict keys
new_state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}

# Initialize the model architecture
model = mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)  # Adjust output classes
model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()

print("Model loaded successfully with modified state_dict and ready for testing.")

# Step 5: Load Test Dataset
test_dataset = CustomDataset(root_dir='/kaggle/input/vista-25/dataset/dataset/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

# Step 6: TTA Function for Inference
def tta_inference(model, dataloader, device):
    model.eval()
    tta_predictions = []
    image_ids = []
    with torch.no_grad():
        for images, ids in dataloader:
            if images is None:
                continue
            images = images.to(device)
            outputs = []
            for flip in [False, True]:  # Horizontal flip
                augmented_images = torch.flip(images, dims=[3]) if flip else images
                outputs.append(torch.softmax(model(augmented_images), dim=1))
            averaged_outputs = torch.mean(torch.stack(outputs), dim=0)[:, 1]  # Probability of class 1
            tta_predictions.extend(averaged_outputs.cpu().numpy())
            image_ids.extend(ids)
    return tta_predictions, image_ids

# Step 7: Perform TTA Inference
tta_predictions, image_ids = tta_inference(model, test_loader, device)

# Step 8: Save Predictions to CSV
submission = pd.DataFrame({'image_id': image_ids, 'label': tta_predictions})
submission.to_csv('mobilenetv2 5.csv', index=False)
print("Submission file 'mobilenetv2.csv' created with TTA.")

<ipython-input-1-4095d020c74a>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location=device)


Model loaded successfully with modified state_dict and ready for testing.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (121554000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (143040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Submission file 'mobilenetv2.csv' created with TTA.
